## Wallaroo Admin Dashboard Metrics Retrieval Tutorial

The following tutorial demonstrates using the Wallaroo MLOps API to retrieve Wallaroo metrics data.  These requests are compliant with Prometheus API endpoints.  

This tutorial lists the metrics queries available and demonstrates how to perform each of the queries.

### Prerequisites

This tutorial assumes the following:

* A Wallaroo Ops environment is installed.
* The Wallaroo SDK is installed.  These examples use the Wallaroo SDK to generate the initial inferences information for the metrics requests.

## Inference Data Generation

This part of the tutorial generates the inference results used for the rest of the tutorial.

### Import libraries

The first step is to import the libraries required.

In [114]:
import json
import numpy as np
import pandas as pd

import pytz
import datetime

import requests
from requests.auth import HTTPBasicAuth

import wallaroo

### Connect to the Wallaroo Instance

A connection to Wallaroo is established via the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [ ]:
wl = wallaroo.Client()



In [73]:
model_name = "ccfraud-model"
model_file_name = "./models/ccfraud.onnx"


The following queries are available for resource consumption.  Note where each request the `/v1/metrics/api/v1/query` endpoint.

| Query Name | Description | Example Query | 
|---|---|---|
| Total CPU Requested |   Number of CPUs requested in the Wallaroo cluster |`sum(wallaroo_kube_pod_resource_requests{resource="cpu"})` |
| Total CPU allocated | Total number of available CPUs in the Wallaroo cluster |`sum(kube_node_status_capacity{resource="cpu"})` | 
| Total GPU Requested | Number of GPUs requested in the Wallaroo cluster |`sum(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu\|qualcomm.com/qaic"})` | 
| Total GPU Allocated | Total number of available GPUs in the Wallaroo cluster |`sum(kube_node_status_capacity{resource=~"nvidia_com_gpu\|qualcomm_com_qaic"})` | 
| Total Memory Requested | Amount of memory requested in the Wallaroo cluster. | `sum(wallaroo_kube_pod_resource_requests{resource="memory"})` | 
| Total Memory Allocated | Total amount of memory available in the Wallaroo cluster. | `sum(kube_node_status_capacity{resource="memory"})` |
| Total Inference Log Storage used | Amount of inference log storage used. | `kubelet_volume_stats_used_bytes{persistentvolumeclaim="plateau-managed-disk"}` |
| Total Inference Log Storage allocated | Total amount of inference log storage available. | `kubelet_volume_stats_capacity_bytes{persistentvolumeclaim="plateau-managed-disk"}` |
| Total Artifact Storage used | Amount of model and orchestration artifact storage used. | `kubelet_volume_stats_capacity_bytes{persistentvolumeclaim="minio"}` |
| Total Artifact Storage allocated | Total amount of model and orchestration artifact storage available. | `kubelet_volume_stats_used_bytes{persistentvolumeclaim="minio"}` |
| Average GPU usage over time | Average GPU usage over the defined time range in the Wallaroo cluster. | `avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu\|qualcomm.com/qaic"})[{duration}] {offset})` |
| Average GPU requested over time | Average number of GPU requested over the defined time range in the Wallaroo cluster |  `avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu\|qualcomm.com/qaic"})[{duration}] {offset})` |
| Average CPU usage over time | Average CPU usage over the defined time range in the Wallaroo cluster. | `avg_over_time(sum(wallaroo_kube_pod_resource_usage{resource=”cpu”})[{duration}] {offset})` |
|  Average CPU requested over time | Average CPU requests over the defined time range in the Wallaroo cluster | `avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource="cpu"})[{duration}] {offset})` |
| Average Memory usage over time | Average memory usage over the defined time range in the Wallaroo cluster. | `avg_over_time(sum(wallaroo_kube_pod_resource_usage{resource="memory"})[{duration}] {offset})` |
| Average Memory requests over time | Average memory requests over the defined time range in the Wallaroo cluster. | `avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource="memory"})[{duration}] {offset})` |
| Average pipelines CPU usage over time | Average CPU usage over the defined time range for an individual Wallaroo pipeline. | `avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_usage{resource="cpu"})[{duration}] {offset})` |
| Average pipelines CPU requested over time | Average number of CPUs requested over the defined time range for an individual Wallaroo pipeline. | `avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_requests{resource="cpu"})[{duration}] {offset})` |
| Average pipelines GPU usage over time | Average GPU usage over the defined time range for an individual Wallaroo pipeline. | `avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu\|qualcomm.com/qaic"})[{duration}] {offset})` |
| Average pipelines GPU requested over time | Average number of GPUs requested over the defined time range for an individual Wallaroo pipeline. | `avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu\|qualcomm.com/qaic"})[{duration}] {offset})` |
| Average pipelines Mem usage over time | Average memory usage over the defined time range for an individual Wallaroo pipeline. | `avg_over_time(sum by(namespace) (wallaroo_kube_pod_resource_usage{resource="memory"})[{duration}] {offset})` |
| Average pipelines Mem requested over time | Average amount of memory requested over the defined time range for an individual Wallaroo pipeline. | `avg_over_time(sum by (namespace)(wallaroo_kube_pod_resource_requests{resource="memory"})[{duration}] {offset})` |
| Pipeline inference log storage | Inference log storage used at the end of the defined time range for an individual Wallaroo pipeline |  `sum by(topic) (topic_bytes@{timestamp})` |

### Metrics for a Specified Time Range in the Past

For queries that retrieve metric data between a range of dates in the past, the following example demonstrates how to use start date, end date, and the offset.

This example uses three variables parameterized and inserted using the Python variable string replacement method:

* `date_start`: The date starting the metric analysis period.
* `date_end`: The end date of the metric analysis period.
* `current_time`: The current time.

These values are then converted into the following:

* `duration`: The amount of time in seconds between `date_start` and `date_end`.
* `offset`: The amount of time in seconds between `date_start` and `current_time`.

The following example show retrieving the average CPU usage over a period of time for the dates 12/1/2025 to 12/3/2025.

```bash
# this is the URL to get this metric
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"
# Retrieve the token 
headers = wl.auth.auth_header()

data_start = selected_timezone.localize(datetime.datetime(2025, 12, 1, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 12, 3, 15, 59, 59))
current_time = datetime.datetime.now(selected_timezone)

duration = int((data_end-data_start).total_seconds())
offset = int((current_time-data_start).total_seconds())


query_avg_cpu_usage = f'avg_over_time(sum(wallaroo_kube_pod_resource_usage{{resource="cpu"}})[{duration}s:] offset {offset}s)'

#request parameters
params_avg_cpu_usage = {
    'query': query_avg_cpu_usage,
}

response_avg_cpu_usage = requests.get(query_url, headers=headers, params=params_avg_cpu_usage)


if response_avg_cpu_usage.status_code == 200:
    print("Average CPU usage over time:")
    display(response_avg_cpu_usage.json())
else:
    print("Failed to fetch Avg CPU usage data:", response_avg_cpu_usage.status_code, response_avg_cpu_usage.text)
```


### Total CPU Requested

* Total CPU Requested
* query 
* `sum(wallaroo_kube_pod_resource_requests{resource="cpu"})`
* Number of CPUs requested in the Wallaroo cluster


In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()


query = 'sum(wallaroo_kube_pod_resource_requests{resource="cpu"})'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {}, 'value': [1764866061.844, '14.406']}]}}

### Total CPU allocated

* Total CPU allocated
* query
* `sum(kube_node_status_capacity{resource="cpu"})`
* Total number of available CPUs in the Wallaroo cluster


In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()


query = 'sum(kube_node_status_capacity{resource="cpu"})'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {}, 'value': [1764866062.086, '48']}]}}

### Total GPU Requested

* Total GPU Requested
* query
* `sum(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu|qualcomm.com/qaic"})`
* Number of GPUs requested in the Wallaroo cluster


In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()


query = 'sum(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu|qualcomm.com/qaic"})'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {}, 'value': [1764866062.314, '2']}]}}

### Total GPU Allocated

* Total GPU Allocated
* query
* `sum(kube_node_status_capacity{resource=~"nvidia_com_gpu|qualcomm_com_qaic"})` 
* Total number of available GPUs in the Wallaroo cluster


In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()


query = 'sum(kube_node_status_capacity{resource=~"nvidia_com_gpu|qualcomm_com_qaic"})'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {}, 'value': [1764866062.545, '5']}]}}

### Total Memory Requested

* Total Memory Requested
* query
* `sum(wallaroo_kube_pod_resource_requests{resource="memory"})`
* Amount of memory requested in the Wallaroo cluster.


In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()


query = 'sum(wallaroo_kube_pod_resource_requests{resource="memory"})'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {}, 'value': [1764866062.778, '32220643328']}]}}

### Total Memory Allocated

* Total Memory Allocated
* query
* `sum(kube_node_status_capacity{resource="memory"})`
* Total amount of memory available in the Wallaroo cluster.


In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()


query = 'sum(kube_node_status_capacity{resource="memory"})'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {}, 'value': [1764866062.982, '197850009600']}]}}

### Total Inference Log Storage used

* Total Inference Log Storage used
* query
* `kubelet_volume_stats_used_bytes{persistentvolumeclaim="plateau-managed-disk"}`
* Amount of inference log storage used.



In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()


query = 'kubelet_volume_stats_used_bytes{persistentvolumeclaim="plateau-managed-disk"}'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'__name__': 'kubelet_volume_stats_used_bytes',
     'beta_kubernetes_io_arch': 'amd64',
     'beta_kubernetes_io_instance_type': 'e2-standard-8',
     'beta_kubernetes_io_os': 'linux',
     'cloud_google_com_gke_boot_disk': 'pd-balanced',
     'cloud_google_com_gke_container_runtime': 'containerd',
     'cloud_google_com_gke_cpu_scaling_level': '8',
     'cloud_google_com_gke_logging_variant': 'DEFAULT',
     'cloud_google_com_gke_max_pods_per_node': '110',
     'cloud_google_com_gke_memory_gb_scaling_level': '32',
     'cloud_google_com_gke_nodepool': 'persistent',
     'cloud_google_com_gke_os_distribution': 'cos',
     'cloud_google_com_gke_provisioning': 'standard',
     'cloud_google_com_gke_stack_type': 'IPV4',
     'cloud_google_com_machine_family': 'e2',
     'cloud_google_com_private_node': 'false',
     'failure_domain_beta_kubernetes_io_region': 'us-central1',
     'failure_domain_beta_kubernete

### Total Inference Log Storage allocated

* Total Inference Log Storage allocated
* query
* `kubelet_volume_stats_capacity_bytes{persistentvolumeclaim="plateau-managed-disk"}`
* Total amount of inference log storage available.


In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()


query = 'kubelet_volume_stats_capacity_bytes{persistentvolumeclaim="plateau-managed-disk"}'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'__name__': 'kubelet_volume_stats_capacity_bytes',
     'beta_kubernetes_io_arch': 'amd64',
     'beta_kubernetes_io_instance_type': 'e2-standard-8',
     'beta_kubernetes_io_os': 'linux',
     'cloud_google_com_gke_boot_disk': 'pd-balanced',
     'cloud_google_com_gke_container_runtime': 'containerd',
     'cloud_google_com_gke_cpu_scaling_level': '8',
     'cloud_google_com_gke_logging_variant': 'DEFAULT',
     'cloud_google_com_gke_max_pods_per_node': '110',
     'cloud_google_com_gke_memory_gb_scaling_level': '32',
     'cloud_google_com_gke_nodepool': 'persistent',
     'cloud_google_com_gke_os_distribution': 'cos',
     'cloud_google_com_gke_provisioning': 'standard',
     'cloud_google_com_gke_stack_type': 'IPV4',
     'cloud_google_com_machine_family': 'e2',
     'cloud_google_com_private_node': 'false',
     'failure_domain_beta_kubernetes_io_region': 'us-central1',
     'failure_domain_beta_kuber

### Total Artifact Storage used

* Total Artifact Storage used
* query
* `kubelet_volume_stats_capacity_bytes{persistentvolumeclaim="minio"}`
* Amount of model and orchestration artifact storage used.



In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()


query = 'kubelet_volume_stats_capacity_bytes{persistentvolumeclaim="minio"}'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'__name__': 'kubelet_volume_stats_capacity_bytes',
     'beta_kubernetes_io_arch': 'amd64',
     'beta_kubernetes_io_instance_type': 'e2-standard-8',
     'beta_kubernetes_io_os': 'linux',
     'cloud_google_com_gke_boot_disk': 'pd-balanced',
     'cloud_google_com_gke_container_runtime': 'containerd',
     'cloud_google_com_gke_cpu_scaling_level': '8',
     'cloud_google_com_gke_logging_variant': 'DEFAULT',
     'cloud_google_com_gke_max_pods_per_node': '110',
     'cloud_google_com_gke_memory_gb_scaling_level': '32',
     'cloud_google_com_gke_nodepool': 'persistent',
     'cloud_google_com_gke_os_distribution': 'cos',
     'cloud_google_com_gke_provisioning': 'standard',
     'cloud_google_com_gke_stack_type': 'IPV4',
     'cloud_google_com_machine_family': 'e2',
     'cloud_google_com_private_node': 'false',
     'failure_domain_beta_kubernetes_io_region': 'us-central1',
     'failure_domain_beta_kuber

### Total Artifact Storage allocated

* Total Artifact Storage allocated
* query
* `kubelet_volume_stats_used_bytes{persistentvolumeclaim="minio"}`
* Total amount of model and orchestration artifact storage available.


In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()


query = 'kubelet_volume_stats_used_bytes{persistentvolumeclaim="minio"}'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'__name__': 'kubelet_volume_stats_used_bytes',
     'beta_kubernetes_io_arch': 'amd64',
     'beta_kubernetes_io_instance_type': 'e2-standard-8',
     'beta_kubernetes_io_os': 'linux',
     'cloud_google_com_gke_boot_disk': 'pd-balanced',
     'cloud_google_com_gke_container_runtime': 'containerd',
     'cloud_google_com_gke_cpu_scaling_level': '8',
     'cloud_google_com_gke_logging_variant': 'DEFAULT',
     'cloud_google_com_gke_max_pods_per_node': '110',
     'cloud_google_com_gke_memory_gb_scaling_level': '32',
     'cloud_google_com_gke_nodepool': 'persistent',
     'cloud_google_com_gke_os_distribution': 'cos',
     'cloud_google_com_gke_provisioning': 'standard',
     'cloud_google_com_gke_stack_type': 'IPV4',
     'cloud_google_com_machine_family': 'e2',
     'cloud_google_com_private_node': 'false',
     'failure_domain_beta_kubernetes_io_region': 'us-central1',
     'failure_domain_beta_kubernete

### Average GPU usage over time

* Average GPU usage over time
* Endpoint: `query`
* `avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu|qualcomm.com/qaic"})[{duration}:] offset {offset})`
* Average GPU usage over the defined time range in the Wallaroo cluster.


In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

current_time = datetime.datetime.now(selected_timezone)

duration = int((data_end-data_start).total_seconds())
offset = int((current_time-data_start).total_seconds())




query = f'avg_over_time(sum(wallaroo_kube_pod_resource_requests{{resource=~"nvidia.com/gpu|qualcomm.com/qaic"}})[{duration}s:] offset {duration}s)'

# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)


#request parameters
params_rps = {
    'query': query
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {}, 'value': [1764973184.648, '2']}]}}

### Average GPU requested over time

* Average GPU requested over time
* query
* `avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu|qualcomm.com/qaic"})[{duration}:] offset {offset})`
* Average number of GPU requested over the defined time range in the Wallaroo cluster


In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

current_time = datetime.datetime.now(selected_timezone)

duration = int((data_end-data_start).total_seconds())
offset = int((current_time-data_start).total_seconds())


query = f'avg_over_time(sum(wallaroo_kube_pod_resource_requests{{resource=~"nvidia.com/gpu|qualcomm.com/qaic"}})[{duration}s:] offset {offset}s)'

#request parameters
params_rps = {
    'query': query
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {}, 'value': [1764973243.038, '1']}]}}

### Average CPU usage over time

* Average CPU usage over time
* query
* `avg_over_time(sum(wallaroo_kube_pod_resource_usage{resource="cpu"})[{duration}:] offset {offset})`
* Average CPU usage over the defined time range in the Wallaroo cluster.


In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

current_time = datetime.datetime.now(selected_timezone)

duration = int((data_end-data_start).total_seconds())
offset = int((current_time-data_start).total_seconds())


query = f'avg_over_time(sum(wallaroo_kube_pod_resource_usage{{resource="cpu"}})[{duration}s:] offset {offset}s)'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {},
    'value': [1764973280.342, '0.12934791422175926']}]}}

### Average CPU requested over time

* Average CPU requested over time
* query
* `avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource="cpu"})[{duration}:] offset {offset})`
* Average CPU requests over the defined time range in the Wallaroo cluster


In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

current_time = datetime.datetime.now(selected_timezone)

duration = int((data_end-data_start).total_seconds())
offset = int((current_time-data_start).total_seconds())


query = f'avg_over_time(sum(wallaroo_kube_pod_resource_requests{{resource="cpu"}})[{duration}s:] offset {offset}s)'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {}, 'value': [1764973311.064, '7.720675925925926']}]}}

### Average Memory usage over time

* Average Memory usage over time
* query
* `avg_over_time(sum(wallaroo_kube_pod_resource_usage{resource="memory"})[{duration}:] offset {offset})`
* Average memory usage over the defined time range in the Wallaroo cluster.


In [105]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

current_time = datetime.datetime.now(selected_timezone)

duration = int((data_end-data_start).total_seconds())
offset = int((current_time-data_start).total_seconds())

query = f'avg_over_time(sum(wallaroo_kube_pod_resource_usage{{resource="memory"}})[{duration}s:] offset {offset}s)'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {}, 'value': [1764973398.031, '6001859973.583539']}]}}

### Average Memory requests over time

* Average Memory requests over time
* query
* `avg_over_time(sum(wallaroo_kube_pod_resource_requests{resource="memory"})[{duration}:] offset {offset})`
* Average memory requests over the defined time range in the Wallaroo cluster.


In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

current_time = datetime.datetime.now(selected_timezone)

duration = int((data_end-data_start).total_seconds())
offset = int((current_time-data_start).total_seconds())


query = f'avg_over_time(sum(wallaroo_kube_pod_resource_requests{{resource="memory"}})[{duration}s:] offset {offset}s)'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {}, 'value': [1764973426.54, '18013261854.34074']}]}}

### Average pipelines CPU usage over time

* Average pipelines CPU usage over time
* query
* `avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_usage{resource="cpu"})[{duration}:] offset {offset})`
* Average CPU usage over the defined time range for an individual Wallaroo pipeline.


In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

current_time = datetime.datetime.now(selected_timezone)

duration = int((data_end-data_start).total_seconds())
offset = int((current_time-data_start).total_seconds())


query = f'avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_usage{{resource="cpu"}})[{duration}s:] offset {offset}s)'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'namespace': 'tinyllama-openai-414'},
    'value': [1764973580.279, '0.00932672794025501']},
   {'metric': {'namespace': 'wallaroo'},
    'value': [1764973580.279, '0.10730162002134773']},
   {'metric': {'namespace': 'whisper-hf-byop-jcw-48'},
    'value': [1764973580.279, '0.011238805135648148']},
   {'metric': {'namespace': 'whisper-hf-byop-replicatest-53'},
    'value': [1764973580.279, '0.00949052053616255']}]}}

### Average pipelines CPU requested over time

* Average pipelines CPU requested over time
* query
* `avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_requests{resource="cpu"})[{duration}:] offset {offset})`
* Average number of CPUs requested over the defined time range for an individual Wallaroo pipeline.


In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

current_time = datetime.datetime.now(selected_timezone)

duration = int((data_end-data_start).total_seconds())
offset = int((current_time-data_start).total_seconds())


query = f'avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_requests{{resource="cpu"}})[{duration}s:] offset {offset}s)'

#request parameters
params_rps = {
    'query': query
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'namespace': 'tinyllama-openai-414'},
    'value': [1764973599.822, '0.10392870134594398']},
   {'metric': {'namespace': 'wallaroo'},
    'value': [1764973599.822, '3.2560000000000002']},
   {'metric': {'namespace': 'whisper-hf-byop-jcw-48'},
    'value': [1764973599.822, '4.35']},
   {'metric': {'namespace': 'whisper-hf-byop-replicatest-53'},
    'value': [1764973599.822, '0.1']}]}}

### Average pipelines GPU usage over time

* Average pipelines GPU usage over time
* query
* `avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu\|qualcomm.com/qaic"})[{duration}:] offset {offset})`
* Average GPU usage over the defined time range for an individual Wallaroo pipeline.



In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

current_time = datetime.datetime.now(selected_timezone)

duration = int((data_end-data_start).total_seconds())
offset = int((current_time-data_start).total_seconds())


query = f'avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_requests{{resource=~"nvidia.com/gpu|qualcomm.com/qaic"}})[{duration}s:] offset {offset}s)'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'namespace': 'whisper-hf-byop-jcw-48'},
    'value': [1764973624.499, '1']}]}}

### Average pipelines GPU requested over time

* Average pipelines GPU requested over time
* query
* `avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_requests{resource=~"nvidia.com/gpu|qualcomm.com/qaic"})[{duration}:] offset {offset})`
* Average number of GPUs requested over the defined time range for an individual Wallaroo pipeline. |


In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

current_time = datetime.datetime.now(selected_timezone)

duration = int((data_end-data_start).total_seconds())
offset = int((current_time-data_start).total_seconds())


query = f'avg_over_time(sum by(namespace)(wallaroo_kube_pod_resource_requests{{resource=~"nvidia.com/gpu|qualcomm.com/qaic"}})[{duration}s:] offset {offset}s)'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'namespace': 'whisper-hf-byop-jcw-48'},
    'value': [1764973660.241, '1']}]}}

### Average pipelines Mem usage over time

* Average pipelines Mem usage over time
* query
* `avg_over_time(sum by(namespace) (wallaroo_kube_pod_resource_usage{resource="memory"})[{duration}:] offset {offset})`
* Average memory usage over the defined time range for an individual Wallaroo pipeline. |


In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

current_time = datetime.datetime.now(selected_timezone)

duration = int((data_end-data_start).total_seconds())
offset = int((current_time-data_start).total_seconds())


query = f'avg_over_time(sum by(namespace) (wallaroo_kube_pod_resource_usage{{resource="memory"}})[{duration}s:] offset {offset}s)'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'namespace': 'tinyllama-openai-414'},
    'value': [1764973697.645, '16573120.303096538']},
   {'metric': {'namespace': 'wallaroo'},
    'value': [1764973697.645, '5039891965.050206']},
   {'metric': {'namespace': 'whisper-hf-byop-jcw-48'},
    'value': [1764973697.645, '940454186.7720164']},
   {'metric': {'namespace': 'whisper-hf-byop-replicatest-53'},
    'value': [1764973697.645, '19173635.792592593']}]}}

### Average pipelines Mem requested over time

* Average pipelines Mem requested over time
* query
* `avg_over_time(sum by (namespace)(wallaroo_kube_pod_resource_requests{resource="memory"})[{duration}:] offset {offset})`
* Inference log storage used at the end of the defined time range for an individual Wallaroo pipeline


In [ ]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

current_time = datetime.datetime.now(selected_timezone)

duration = int((data_end-data_start).total_seconds())
offset = int((current_time-data_start).total_seconds())


query = f'avg_over_time(sum by (namespace)(wallaroo_kube_pod_resource_requests{{resource="memory"}})[{duration}s:] offset {offset}s)'

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)


if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'namespace': 'tinyllama-openai-414'},
    'value': [1764973721.216, '139498425.49508196']},
   {'metric': {'namespace': 'wallaroo'},
    'value': [1764973721.216, '8061452288']},
   {'metric': {'namespace': 'whisper-hf-byop-jcw-48'},
    'value': [1764973721.216, '9797894144']},
   {'metric': {'namespace': 'whisper-hf-byop-replicatest-53'},
    'value': [1764973721.216, '134217728']}]}}

### Pipeline inference log storage

* Pipeline inference log storage
* query
* `avg_over_time(sum by(topic) (topic_bytes)[{duration}:] offset {offset})`
* Inference log storage used at the end of the defined time range for an individual Wallaroo pipeline

In [119]:
# this is the URL to get prometheus metrics
query_url = f"{wl.api_endpoint}/v1/metrics/api/v1/query"

# Retrieve the token 
headers = wl.auth.auth_header()

# this will also format the timezone in the parsing section
timezone = "US/Mountain"
selected_timezone = pytz.timezone(timezone)
data_start = selected_timezone.localize(datetime.datetime(2025, 11, 24, 13, 0, 0))
data_end = selected_timezone.localize(datetime.datetime(2025, 11, 25, 15, 59, 59))

# This example uses the data_end period to get sizes at a specific point in time
timestamp = int(data_end.timestamp())

# Other example: now() to retrieve sizes for the current time
# timestamp = int(datetime.datetime.now(selected_timezone).timestamp())

# Get the total size of all the pipeline files by pipeline AND pipeline_version
query = f'sum by(topic) (topic_bytes@{timestamp})'

print(query)

#request parameters
params_rps = {
    'query': query,
}

response = requests.get(query_url, headers=headers, params=params_rps)

if response.status_code == 200:
    print("Query Response:")
    display(response.json())
else:
    print("Failed to fetch query response:", response.status_code, response.text)

sum by(topic) (topic_bytes@1764111599)
Query Response:


{'status': 'success',
 'data': {'resultType': 'vector',
  'result': [{'metric': {'topic': 'workspace-121-pipeline-whisper-hf-byop-jcw-inference'},
    'value': [1765409252.95, '0']},
   {'metric': {'topic': 'workspace-1444-pipeline-dlrm-click-prediction-inference'},
    'value': [1765409252.95, '4282479']},
   {'metric': {'topic': 'workspace-1526-pipeline-ma-consumptionchanges-stage-inference'},
    'value': [1765409252.95, '1148670']},
   {'metric': {'topic': 'workspace-1529-pipeline-rum-assay-nan-jcw-inference'},
    'value': [1765409252.95, '2041174']},
   {'metric': {'topic': 'workspace-1786-pipeline-ai-screening-inference'},
    'value': [1765409252.95, '2817923']},
   {'metric': {'topic': 'workspace-42-pipeline-retail-inv-tracker-edge-obs-inference'},
    'value': [1765409252.95, '3500604']},
   {'metric': {'topic': 'workspace-71-pipeline-assay-demonstration-tutorial-jcw-inference'},
    'value': [1765409252.95, '60325542']},
   {'metric': {'topic': 'workspace-86-pipeline-house-p